In [4]:
# 计算语料库的平均长度

file_path = 'data\wiki1m_for_simcse.txt'

total_length = 0
total_sentences = 0

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        words = line.strip().split()
        total_length += len(words)
        total_sentences += 1

average_length = total_length / total_sentences
print(average_length)


19.432477


In [8]:
# 分词测试
import spacy

# 加载英语语言模型
nlp = spacy.load("en_core_web_sm")

# 分词示例
text = "South Australia (SA)  has a unique position in Australia's history as, unlike the other states which were founded as colonies, South Australia began as a self governing province Many were attracted to this and Adelaide and SA developed as an independent and free thinking state."
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

# # 关键词提取示例
# print("\n关键词提取:")
# keywords = [token.text for token in doc if not token.is_stop and token.is_alpha]

# # 选择关键词列表中的第一个词作为最关键的词
# most_important_word = keywords[0] if keywords else None
# print(most_important_word)


South Australia NORP
SA ORG
Australia GPE
colonies GPE
South Australia GPE
Adelaide ORG
SA ORG
